In [27]:
import promo_functions
import xml.etree.ElementTree as Tree



def populatePromoDict():
	global promo_dict
	promo_dict = {}
	for item in item_names:
		promo_nodes = inventory.findall("./item[@name='%s']/promo" % item)
		if (len(promo_nodes) > 0):
			promo_dict[item] = []
			for node in promo_nodes:
				nth = node.find('./nth')
				name = node.find('./name')
				discount = node.find('./discount')
				func = node.find('./func')
				# nth and (discount or func) are required - name is optional
				if (nth == None):
					continue
				else:
					nth_val = nth.text
					try:
						nth_int_val = int(nth_val)
					except:
						continue
				if (name == None):
					name_val = "Promotion Discount"
				else:
					name_val = name.text
					
				# keep track of either discount of func True
				have_a_promo = False
				if (discount != None):
					discount_val = discount.text
					try:
						discount_float_val = float(discount_val)
						have_a_promo = True
					except:
						discount_float_val = 0
				if (func != None):
					try:
						func_val = eval("promo_functions.%s" % func.text)
						have_a_promo = True
					except:
						func_val = None
				else:
					func_val = None
				
				# don't actually have any kind of promotion - skip this node
				if (not have_a_promo):
					continue
				
				# we should have all legit values to create a Promo instance
				promo = Promo(item, nth_int_val, name_val, discount_float_val, func_val)
				#print("PROMO: %s %d %s %f %s" % (promo.item, promo.nth, promo.name, promo.save, promo.func)
				promo_dict[item].append(promo)

def getRegularCostOfItem(item):
	price_node = inventory.find("./item[@name='%s']/price" % item)
	try:
		cost = float(price_node.text)
	except:
		# not found, so it must be free!
		cost = 0
	return cost

def getPriceAndPromoTuple(item, nth):
	cost = getRegularCostOfItem(item)
	deal = ""
	
	# check for promo
	if (item in promo_dict):
		for promo in promo_dict[item]:
			if (promo.promoDoesApply(item, nth)):
				cost += promo.discount(item, nth)
				deal = promo.name

	return (cost, deal)
		
def printReceipt(purchase):
	total = 0
	print('\n')
	print('{:<20}'.format('Purchase'), '{:>8}'.format('Price'), '{:>8}'.format('Total'), ' '*3, "Promotion")
	print('-'*20, '{:>8}'.format('-'*5), '{:>8}'.format('-'*5), ' '*3, '-'*9)
	# tuple has (item name, item cost, item promo)
	for tuple in purchase:
		total += tuple[1]
		print('{:.<20}'.format(tuple[0]), '{:>8}'.format('{:.2f}'.format(tuple[1])), '{:>8}'.format('{:.2f}'.format(total)), ' '*3, tuple[2])
	
	print("\nTotal: £", '{0:.2f}'.format(total))
        
class Promo:
	def __init__(self, item_type, this_many, promo_name, discount=0, expr=None):
		self.item = item_type
		self.nth = this_many
		self.name = promo_name
		self.save = discount
		self.func = expr
		
	def promoDoesApply(self, item, num):
		return num % self.nth == 0

	def discount(self, item, num):
		cost = getRegularCostOfItem( item)
		discount = 0

		if (self.promoDoesApply(item, num)):
			#apply the promo
			if (self.save > 0):
				discount = - self.save
			elif (self.func != None):
				try:
					# all promo functions are expected to have this argument list: item, cost, nth and return the new promo cost
					discount = self.func(self.item, cost, self.nth)
				except:
					#it didn't work, leave as is
					pass
		return discount

#used for debugging	
def printDailyDeals():
	global promo_dict
	label = "Today's Promotions:"
	print("%s\n" % label, '-'*len(label))
	for item, list in promo_dict.items():
		for promo in list:
			print("%s: %s" % (promo.item, promo.name))

# make a list of the produce inventory item names


def printHelp():
	printDailyDeals()
	# print(the inventory choices list
	label = "Inventory"
	print("\n%s\n" % label, '-'*len(label))
	for item in item_names:
		print("-> ", item)
	print("Enter return to complete your transaction and print a receipt!\n")

if __name__ =='__main__':
	#read the inventory data file
	try:
		tree = Tree.parse('inventory.xml')
		inventory = tree.getroot()
	except:
		print("Error parsing the inventory.xml data file - exiting program")
		exit()
	purchased = []

	# total spent
	total = 0
	# number of items purchased indexed by item
	totals_for_items = {}
	item_names = []
	nodes = inventory.findall("./item[@name]")
	for node in nodes:
		item_names.append( node.attrib['name'])
	num_items = len(item_names)

	promo_dict = {}
	populatePromoDict()
	input_product_items = input()
	input_list = input_product_items.split(" ")
	for product_item in input_list:
		if (product_item in item_names):
			# keep track of the number of each item bought
			if (not product_item in totals_for_items):
				totals_for_items[product_item] = 0

			num_bought = totals_for_items[product_item] + 1
			totals_for_items[product_item] = num_bought

			# expecting (price, promo text)
			tuple = getPriceAndPromoTuple(product_item, num_bought)
			total += tuple[0]
			purchased.append((product_item, tuple[0], tuple[1]))
			# print('{:<10}'.format(product_item), '{:>8}'.format('{:.2f}'.format(tuple[0])), '{:>8}'.format('{:.2f}'.format(total)),' '*3, tuple[1]
		elif (product_item == "help"):
			printHelp()
		elif (len(product_item) == 0):
			break
		else:
			# invalid produce, ignore
			pass

	# print(the receipt
	printReceipt(purchased)



A B C A B D E A


Purchase                Price    Total     Promotion
--------------------    -----    -----     ---------
A...................    50.00    50.00     
B...................    30.00    80.00     
C...................    20.00   100.00     
A...................    50.00   150.00     
B...................    15.00   165.00     2 for £45
D...................    15.00   180.00     
E...................     3.00   183.00     
A...................    30.00   213.00     3 for £130

Total: £ 213.00
